ECE 8803 Final Project

In [5]:
import numpy
import pandas as pd
import os

root = os.getcwd()
train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")
trainset = pd.read_csv(train)
testset = pd.read_csv(test)